In [117]:


import pandas as pd

# fead all the files in the folder results
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from io import StringIO
import pickle
import json

# open config

# open the file
with open("config.json", "r") as file:
    config = json.load(file)


folders = os.listdir("Results")

for folder in folders:
    path = "Results/"+folder+"/NeuroSim_Results_Each_Epoch"
    files = os.listdir(path)
    with open("Results/" + folder + "/arch.pkl", 'rb') as file:
        input = pickle.load(file)
    
    
    # create input to numpy array
    input = np.array(input)
    with open(path +"/NeuroSim_Breakdown_Epock_0.csv", 'r') as file:
        csv_content = file.read()
        delimiter = "\n\n"  # Replace this with the appropriate delimiter/pattern
        csv_file = StringIO(csv_content.split(delimiter)[1])
        df = pd.read_csv(csv_file)
        latency = (df[' latency_FW(s)'])[:-1]
        energy = (df[' energy_FW(J)'])[:-1]
        
        latency = np.array(latency)
        energy = np.array(energy)
        print(len(latency) , len(energy) , config["output_size"])
        if len(latency) < config["output_size"]:
            # pad with zeroes
            latency = np.pad(latency, (0, config["output_size"]-len(latency)), 'constant')
            energy = np.pad(energy, (0, config["output_size"]-len(energy)), 'constant')
        print(len(latency) , len(energy) , config["output_size"])
        if not os.path.exists("./dataset.pkl"):
            with open ("./dataset.pkl" , "wb") as file:
                pickle.dump([(input , (latency , energy))] , file)
        else : 
            with open ("./dataset.pkl" , "rb") as file:
                
                existing_data = pickle.load(file)
            with open ("./dataset.pkl" , "wb") as file:
                existing_data.append((input , (latency , energy)))
                pickle.dump(existing_data , file)
            


8 8 9
9 9 9
1 1 9
9 9 9
9 9 9
9 9 9
5 5 9
9 9 9
6 6 9
9 9 9
5 5 9
9 9 9
3 3 9
9 9 9
4 4 9
9 9 9
4 4 9
9 9 9
7 7 9
9 9 9


In [118]:
import torch 
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader , TensorDataset
import numpy as np
import pickle

with open ("./dataset.pkl" , "rb") as file:
    data = pickle.load(file)

data_loader = DataLoader(data , batch_size = 1 , shuffle = False)

In [119]:
for i in data_loader:
    X = i[0]
    Y = i[1]
    print(Y[0])
    break

tensor([[329.5980,  85.6604,  10.0996,   2.0684,   0.5522,   0.4129,   0.6388,
           0.4795,   0.0000]], dtype=torch.float64)


In [120]:
# create a model

class Model(nn.Module):
    def __init__(self , input_size ,hidden_size, output_size):
        super(Model , self).__init__()
        self.fc1 = nn.Linear(input_size , hidden_size)
        self.fc2 = nn.Linear(hidden_size , hidden_size)
        self.fc3 = nn.Linear(hidden_size , output_size)
    def forward(self , x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        return x


model = Model(config["embedding_size"] , config["hidden_size"] ,2* config["output_size"])
optimizer = optim.SGD(model.parameters() , lr = 0.001 , momentum = 0.9)
criterion = nn.MSELoss()


In [121]:
for epoch in range(1000):
    model.train()
    for i in data_loader:
        X = torch.tensor(i[0]).float()
        (Y1 , Y2) = i[1]
        Y1 = torch.tensor(Y1).float().squeeze(0)
        Y2 = torch.tensor(Y2).float().squeeze(0)
        optimizer.zero_grad()
        output = model(X)
        print(output)
        labels = torch.stack([Y1, Y2]).flatten()
        loss = criterion(output , labels)
        loss.backward()
        optimizer.step()
        break
print(loss.item())

/tmp/ipykernel_3481871/3579252344.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(i[0]).float()
/tmp/ipykernel_3481871/3579252344.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Y1 = torch.tensor(Y1).float().squeeze(0)
/tmp/ipykernel_3481871/3579252344.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Y2 = torch.tensor(Y2).float().squeeze(0)
/raid/home/gourishanker/.local/lib/python3.8/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([18])) that is different to the input s

tensor([[ 0.0000,  5.3917,  0.0000,  0.0000,  0.0000,  0.0000,  0.1278,  0.0000,
          0.0000,  0.0000,  0.0000,  4.5929, 38.5357,  2.0879,  0.0000, 11.1416,
          0.0513, 15.0915]], grad_fn=<ReluBackward0>)
tensor([[     0.0000, 135297.5781,      0.0000,      0.0000,      0.0000,
              0.0000,      0.0000,      0.0000,      0.0000,      0.0000,
              0.0000,      0.0000,      0.0000,      0.0000,      0.0000,
              0.0000,   1224.9421,      0.0000]], grad_fn=<ReluBackward0>)
tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0404, 0.0000, 0.0364,
         0.0691, 0.0000, 0.0487, 0.0000, 0.0000, 0.1239, 0.0000, 0.0000, 0.0000]],
       grad_fn=<ReluBackward0>)
tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0453, 0.0000, 0.1635,
         0.0586, 0.0000, 0.0732, 0.0000, 0.0000, 0.1490, 0.0000, 0.0000, 0.0000]],
       grad_fn=<ReluBackward0>)
tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0512, 0.0000, 0.2775,
         0.049